# The `python-benedict` Library

I actually ran across this library by accident recently, and it is an interesting library that makes accessing and traversing nested dictionaries quite simple, so I thought I'd cover some of its functionality here.

This library is called `python-benedict`, with documentation here:
[https://github.com/fabiocaccamo/python-benedict](https://github.com/fabiocaccamo/python-benedict)

I assume that since the author is Italian, `benedict` must mean something like well-behaved dictionary. If someone knows, leave a comment!

To install the full library, use pip this way:
`pip install "python-benedict[all]"`

This installs the full library. It is possible to only install a subset of the library depending on your needs. This relates to the fact that this library can retrieve dictionary-like objects from various sources, such as CSV, JSON, http requests, S3, and more.

The library seems to be under active development with 1.3k stars, although it seems this is a single developer supporting this, so take that into consideration.

Let's jump right in.

We're going to cover these features:
- instantiating `benedict` objects, from dictionaries, HTTP requests and JSON - it has plenty other options too
- accessing and mutating these `benedict` objects (which are `dict` subclasses) in a variey of ways
- making the objects immutable
- a few traversal and filtering methods - this is where I think this library shines - not just easier element access, but a slew of utility functions for traversing, filtering, grouping, finding, and a whole lot more.

We'll start by defining a nested dictionary:

In [1]:
rectangle = {
    "dimensions": {
        "length": 100,
        "width": 50,
    },
    "center": {
        "x": 10, 
        "y": 5,
    },
    "colors": {
        "edge": {
            "rgb": (100, 100, 0),
            "thickness": 50,
        },
        "interior": {
            "rgb": (0, 255, 255),
            "alpha": 50,
        },
    },
}
 

Now let's say we want to get the `alpha` value for the interior.

Using straight Python, we could do something like this:

In [2]:
rectangle["colors"]["interior"]["alpha"]

50

Let's use `python-benedict` to do the same thing.

In [3]:
from benedict import benedict

The `benedict` object is a subclass of Python's `dict`, so you can simply case a regular dictionary:

In [4]:
rect = benedict(rectangle)

In [5]:
type(rect)

benedict.dicts.benedict

In [6]:
print(rect)

{'dimensions': {'length': 100, 'width': 50}, 'center': {'x': 10, 'y': 5}, 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50}, 'interior': {'rgb': (0, 255, 255), 'alpha': 50}}}


It is now possible to access keys in the nested dictionary in a variety of ways.

The simplest is probably just using dot notation:

In [7]:
rect.colors.interior.alpha

50

We can actually also use this to set values in the `benedict` object:

In [8]:
rect.colors.interior.alpha = 60

In [9]:
rect

{'dimensions': {'length': 100, 'width': 50}, 'center': {'x': 10, 'y': 5}, 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50}, 'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

Note that it "mutated" our original dictionary!

In [10]:
rectangle

{'dimensions': {'length': 100, 'width': 50},
 'center': {'x': 10, 'y': 5},
 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50},
  'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

So, the original dictionary and the `benedict` object are "linked" - keep that in mind.

You could always create a deepcopy of your original dict and use that instead if you do not want this.

Of course, since these objects are `dict` objects:

In [11]:
isinstance(rect, dict)

True

this means they also support the standard dictionary access:

In [12]:
rect["colors"]["interior"]["alpha"]

60

But, with `benedict` objects, you can also specify a path using a variable arg list of string keys:

In [13]:
rect["colors", "interior", "alpha"]

60

You can also use a string path, similar to the dot notation, but as a single string:

In [14]:
rect["colors.interior.alpha"]

60

These variants allow you quite a bit of flexibility when building up paths programmatically to search for objects in your dictionaries.

But of course, this begs the question, what if the dictionary keys contain periods (`.`) in them?

Let's try it out:

In [15]:
data = {
    "a": {
        "a.1": 100,
        "a.2": 200,
    },
    "b": {
        "b.2": 10,
        "b.3": 20,
    }
}

In [16]:
try:
    benedict(data)
except ValueError as ex:
    print(ex)

Key should not contain keypath separator '.', found: 'a.1'.


So, we can't load the dictionary - however, we could specify a different keypath separator.

In this case for example, we could choose to use a pipe character (`|`):

In [17]:
d = benedict(data, keypath_separator="|")

And now we can use this separator when dealing with string paths:

In [18]:
d["a|a.2"]

200

In [19]:
d["a", "a.2"]

200

but what about accessing using attributes?

In [20]:
d.a

{'a.1': 100, 'a.2': 200}

Of course, we seem to lose the ability to access the elements using attributes.

This would not make a while lot of sense:
```python
d.a.a.1
```

Although you could certainly use something like this:

In [21]:
d.a["a.1"]

100

Lists are also supported, and as expected we simply use `[i]` to sepcify the element of the list we want:

In [22]:
data = {
    "a": {
        "a_1": [1, 2, 3],
    }
}

In [23]:
d = benedict(data)

In [24]:
d.a.a_1[-1]

3

This library allows us to "load" the data from a variety of data sources. For example, we could specify a URL to get some JSON data back from some API.

Here I'll use the Starwars API (SWAPI) as an example:

In [25]:
luke = benedict("https://swapi.dev/api/people/1/", format="json")

In [26]:
luke

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}

And we can retrieve (or set) elements via attributes or paths:

In [27]:
luke.films[0]

'https://swapi.dev/api/films/1/'

In [28]:
luke["films[0]"]

'https://swapi.dev/api/films/1/'

Since I am unsure of how this is http call is implemented, especially as far as timeouts, headers, authentication, I would probably not use this, but instead run the HTTP request myself using `requests` so i can fully control what is happening, then use the `from_json()` method provided by `benedict`:

In [29]:
import requests

result = requests.get("https://swapi.dev/api/people/1/")
result.raise_for_status()

data = result.text
data

'{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/14/","https://swapi.dev/api/vehicles/30/"],"starships":["https://swapi.dev/api/starships/12/","https://swapi.dev/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"https://swapi.dev/api/people/1/"}'

In [30]:
luke = benedict.from_json(data)
luke

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}

Or you could of course just use the `json()` method available in `requests` and cast the resulting `dict` to `benedict`.

In [31]:
luke = benedict(result.json())
luke

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}

As we saw earlier, the `benedict` objects are mutable. What we did not see is how we **add** elements to the dictionary:

In [32]:
rectangle

{'dimensions': {'length': 100, 'width': 50},
 'center': {'x': 10, 'y': 5},
 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50},
  'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

In [33]:
rect = benedict(rectangle).clone()

Using standard Python notation wil work just fine:

In [34]:
rect["colors"]["edge"]["alpha"] = 10
rect

{'dimensions': {'length': 100, 'width': 50}, 'center': {'x': 10, 'y': 5}, 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50, 'alpha': 10}, 'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

> Side note: since we made a `clone` of the benedict object, we are no longer "tied" to the original dictionary. We coudl also just have deep copied the original dictionary, and then created a `benedict` object from it.

In [35]:
rectangle

{'dimensions': {'length': 100, 'width': 50},
 'center': {'x': 10, 'y': 5},
 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50},
  'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

But what about using the other access notations?

In [36]:
rect = benedict(rectangle).clone()

In [37]:
rect.colors.edge.alpha = 10

In [38]:
rect

{'dimensions': {'length': 100, 'width': 50}, 'center': {'x': 10, 'y': 5}, 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50, 'alpha': 10}, 'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

## Dictionary Methods

Now let's look at some of the really interesting aspects of this library.

Let's load some data of all the people in the Star Wars API:

In [39]:
response = requests.get("https://swapi.dev/api/people")
response.raise_for_status()
data = response.json()

people = benedict(data)
people

{'count': 82, 'next': 'https://swapi.dev/api/people/?page=2', 'previous': None, 'results': [{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}, {'name': 'C-3PO', 'height': '167', 'mass': '75', 'hair_color': 'n/a', 'skin_color': 'gold', 'eye_color': 'yellow', 'birth_year': '112BBY', 'gender': 'n/a', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https:/

### Traversals

As you can see we have some nested dictionaries here.

We can easily traverse each entry of the dictionary using the `traverse()` method. This method needs to be provided a **callable** that will receive three arguments: the (possibly nested) dictionary, and within that dictionary the key and the corresponding value.

In [40]:
def process_item(d, key, val):
    print(f"{key} -> {val}")

In [41]:
people.traverse(process_item)

count -> 82
next -> https://swapi.dev/api/people/?page=2
previous -> None
results -> [{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}, {'name': 'C-3PO', 'height': '167', 'mass': '75', 'hair_color': 'n/a', 'skin_color': 'gold', 'eye_color': 'yellow', 'birth_year': '112BBY', 'gender': 'n/a', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi

### Filtering

We can also easily filter the dictionary, all we need is to pass the `filter()` method a **predicate** function (a function that returns `True` to include the item, `False` otherwise, that receives the key and value as arguments. Basically this means we can look for specific key and/or value combinations in the dictionary.

Let's say we just want to see the names of the people in our data set:

In [42]:
data = people.results

In [43]:
data[0]

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}

In [44]:
[d.filter(lambda k, v: k == "name") for d in data]

[{'name': 'Luke Skywalker'},
 {'name': 'C-3PO'},
 {'name': 'R2-D2'},
 {'name': 'Darth Vader'},
 {'name': 'Leia Organa'},
 {'name': 'Owen Lars'},
 {'name': 'Beru Whitesun lars'},
 {'name': 'R5-D4'},
 {'name': 'Biggs Darklighter'},
 {'name': 'Obi-Wan Kenobi'}]

Do note that this filter method is not recursive, i.e. the keys and values received apply only to the top level of the object being filtered.

## Matching

For a nested approach, to find something of interest, we can use the `match()` method with a wildcard (`*`) to indicate we want any path:

In [45]:
people.match("*.name")

['Luke Skywalker',
 'C-3PO',
 'R2-D2',
 'Darth Vader',
 'Leia Organa',
 'Owen Lars',
 'Beru Whitesun lars',
 'R5-D4',
 'Biggs Darklighter',
 'Obi-Wan Kenobi']

You can even use the wildcard for list indexes - here we are going to match against every element of the `results` key:

In [46]:
people.match("results[*].name")

['Luke Skywalker',
 'C-3PO',
 'R2-D2',
 'Darth Vader',
 'Leia Organa',
 'Owen Lars',
 'Beru Whitesun lars',
 'R5-D4',
 'Biggs Darklighter',
 'Obi-Wan Kenobi']

## Flattening Dictionaries

Something that is sometimes of interest, is flattening the dictionary. We can use the `flatten()` method to do this:

In [47]:
 rect

{'dimensions': {'length': 100, 'width': 50}, 'center': {'x': 10, 'y': 5}, 'colors': {'edge': {'rgb': (100, 100, 0), 'thickness': 50, 'alpha': 10}, 'interior': {'rgb': (0, 255, 255), 'alpha': 60}}}

In [48]:
rect.flatten()

{'dimensions_length': 100, 'dimensions_width': 50, 'center_x': 10, 'center_y': 5, 'colors_edge_rgb': (100, 100, 0), 'colors_edge_thickness': 50, 'colors_edge_alpha': 10, 'colors_interior_rgb': (0, 255, 255), 'colors_interior_alpha': 60}

We can make this a bit more readable by using the `dump()` method:

In [49]:
print(rect.flatten(separator=":").dump())

{
    "center:x": 10,
    "center:y": 5,
    "colors:edge:alpha": 10,
    "colors:edge:rgb": [
        100,
        100,
        0
    ],
    "colors:edge:thickness": 50,
    "colors:interior:alpha": 60,
    "colors:interior:rgb": [
        0,
        255,
        255
    ],
    "dimensions:length": 100,
    "dimensions:width": 50
}


## Key paths

We can also get a list of all the key paths in our dictionary (so, without the values):

In [50]:
rect.keypaths()

['center',
 'center.x',
 'center.y',
 'colors',
 'colors.edge',
 'colors.edge.alpha',
 'colors.edge.rgb',
 'colors.edge.thickness',
 'colors.interior',
 'colors.interior.alpha',
 'colors.interior.rgb',
 'dimensions',
 'dimensions.length',
 'dimensions.width']

We can even have it report back on any indexes from iterables:

In [51]:
people.results[0].keypaths(indexes=True)

['birth_year',
 'created',
 'edited',
 'eye_color',
 'films',
 'films[0]',
 'films[1]',
 'films[2]',
 'films[3]',
 'gender',
 'hair_color',
 'height',
 'homeworld',
 'mass',
 'name',
 'skin_color',
 'species',
 'starships',
 'starships[0]',
 'starships[1]',
 'url',
 'vehicles',
 'vehicles[0]',
 'vehicles[1]']

## Conclusion

As you can see this library has some really interesting functionality, along with quite a bit more I have not covered here. So, give it a try, read the docs, and see if there's something you might find useful in your projects.

Happy Pythoning!